In [ ]:
!pip install insightface onnxruntime-gpu opencv-python albumentations tqdm pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 15.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.6 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp312-cp312-linux_x86_64.whl size=1070141 sha256=ccdb97f44e675cc9f6c9f7339fa0cf30d47ec585ae0c5845045e9fa2a14c9165
  Stored in directory: /root/.cache/pip/wheels/73/3c/e2/6d4815e8a8b33a2006554d65ce0d1f973e768f4c7a222fa675
Successfully built insightface


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Unzipped /content/drive/MyDrive/Sentinel_FaceV1.zip to /content/drive/MyDrive/Sentinel_FaceV1/


In [ ]:
from insightface.app import FaceAnalysis
import cv2, os
from tqdm import tqdm

# Initialize SCRFD face detector
app = FaceAnalysis(name="buffalo_l")  # 'buffalo_l' = SCRFD + ArcFace combo
app.prepare(ctx_id=0, det_size=(640, 640))  # GPU (ctx_id=0), set detection size


download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:05<00:00, 55124.85KB/s]


*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:129 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, SUCCTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; SUCCTYPE = cudaError; std::conditional_t<THRW, void, common::Status> = void] /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:121 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, SUCCTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; SUCCTYPE = cudaError; std::conditional_t<THRW, void, common::Status> = void] CUDA failure 35: CUDA driver version is insufficient for CUDA runtime version ; GPU=-1 ; hostname=886dc57b8071 ; file=/onnxruntime_src/onnxruntime/core/providers/cuda/cuda_execution_provider.cc ; line=282 ; expr=cudaSetDevice(info_.device_id); 

 when using 

In [ ]:
print("Hello")

Hello


In [ ]:
def process_folder(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for img_name in tqdm(os.listdir(input_dir)):
        if not img_name.lower().endswith(('.jpg', '.png', '.jpeg')):
            continue
        img_path = os.path.join(input_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue

        faces = app.get(img)
        if len(faces) == 0:
            continue  # Skip if no face detected

        # Take the largest face (useful for selfies)
        # Add a check to ensure that 'faces' is not empty before finding the max face
        if faces: # Check if faces list is not empty
            face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
        else:
            continue # Skip if no face was detected

        # Aligned face crop from SCRFD output
        aligned = face.crop_face(img)
        if aligned is not None:
            cv2.imwrite(os.path.join(output_dir, img_name), aligned)

In [ ]:
base = "/content/drive/MyDrive/Sentinel_FaceV1/Forgery_Dataset"
process_folder(f"{base}/real", "processed_faces/real")
process_folder(f"{base}/fake", "processed_faces/fake")

dup_base = "/content/drive/MyDrive/Sentinel_FaceV1/Duplication_Dataset"
for person_id in os.listdir(dup_base):
    input_dir = os.path.join(dup_base, person_id)
    output_dir = os.path.join("processed_faces/duplication", person_id)
    process_folder(input_dir, output_dir)

  0%|          | 0/10000 [00:01<?, ?it/s]


TypeError: 'NoneType' object is not callable

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose([
    A.RandomBrightnessContrast(p=0.3),
    A.HorizontalFlip(p=0.5),
    A.Resize(224, 224),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])


In [ ]:
import pandas as pd

df_forgery = pd.read_csv("sentinel_faceV1/Forgery_dataset/train_labels.csv")
df_forgery['path'] = df_forgery.apply(
    lambda x: f"processed_faces/{x['label']}/{x['image_name']}", axis=1
)


In [ ]:
dup_pairs = []
dup_path = "processed_faces/duplication"
for person_id in os.listdir(dup_path):
    imgs = os.listdir(os.path.join(dup_path, person_id))
    for i in range(len(imgs)):
        for j in range(i+1, len(imgs)):
            dup_pairs.append((
                os.path.join(dup_path, person_id, imgs[i]),
                os.path.join(dup_path, person_id, imgs[j]),
                1
            ))


In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    df_forgery, test_size=0.2, stratify=df_forgery['label'], random_state=42
)
